In [1]:
import jijmodeling as jm

In [2]:
# シフト最適化の数理モデル

# シフト希望表
# 1: シフトを希望する
# 0: シフトを希望しない
H = jm.Placeholder("H", ndim=3)

# シフトを組む日数 D, 時間分割数 T, 人数 Ns
D = H.len_at(0, latex="D")
T = H.len_at(1, latex="T")
N = H.len_at(2, latex="N")

# 必要人数
R = jm.Placeholder("R", ndim=2)

# シフト表
# 1: シフトを組む
# 0: シフトを組まない
x = jm.BinaryVar("x", shape=(D, T, N))


problem = jm.Problem(name="shift_optimization")

# 必要人数を満たす制約
d = jm.Element("d", belong_to=(0, D))
t = jm.Element("t", belong_to=(0, T))
problem += jm.Constraint("demand", x[d, t, :].sum() >= R[d, t], forall=[d, t])

# # 夜勤 (t=T-1) の次の日は休みを取る制約
i = jm.Element("i", belong_to=(0, N))
# problem += jm.Constraint("night_shift", x[d+1, t, i] <= x[d, T-1, i], forall=[(d, d < D-1), t, i])

# シフトを希望しない場合はシフトを組まない制約
problem += jm.Constraint("request", x[d, t, i] == 0, forall=[d, t, (i, H[d, t, i] == 0)])

# 目的関数 必要人数の最小化
problem += jm.sum([d, t, i], x[d, t, i])

problem

In [3]:
# データの用意
import numpy as np

def create_data(D, T, N):
    H = np.random.randint(0, 2, (D, T, N))
    R = np.random.randint(0, 3, (D, T))
    # 必要人数を満たすための最小人数になるように調整
    R = np.minimum(R, np.sum(H, axis=2))
    return {
        "H": H,
        "R": R
    }

In [4]:
num_days = 3
num_times = 3
num_people = 2

data = create_data(num_days, num_times, num_people)
interpreter = jm.Interpreter(data)

instance = interpreter.eval_problem(problem)

In [5]:
import ommx_pyscipopt_adapter as ompc

In [6]:
model = ompc.instance_to_model(instance)

In [7]:
model.optimize()
model.getStatus()

'optimal'

In [8]:
solution = ompc.model_to_solution(model, instance)

In [45]:
values = solution.raw.state.entries
deci_vars = {}
for variable in solution.raw.decision_variables:
    if variable.name not in deci_vars:
        deci_vars[variable.name] = {}
    deci_vars[variable.name][tuple(variable.subscripts)] = values[variable.id]


In [46]:
deci_vars

{'x': {(0, 0, 0): 1.0,
  (0, 0, 1): 0.0,
  (0, 1, 0): 1.0,
  (0, 1, 1): 0.0,
  (0, 2, 0): 0.0,
  (0, 2, 1): 1.0,
  (1, 0, 0): 0.0,
  (1, 0, 1): 1.0,
  (1, 1, 0): 0.0,
  (1, 1, 1): 0.0,
  (1, 2, 0): 0.0,
  (1, 2, 1): 0.0,
  (2, 0, 0): 1.0,
  (2, 0, 1): 0.0,
  (2, 1, 0): 1.0,
  (2, 1, 1): 0.0,
  (2, 2, 0): 0.0,
  (2, 2, 1): 1.0}}